In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import building_blocks
from building_blocks import *

from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau

from datasets import EliaSolarDataset
from utils import ConfigSettings

In [2]:
cf = ConfigSettings(config_path='config.ini')

In [3]:
solar_dataset = EliaSolarDataset(
        csv_path=cf.data.file_path,
        datetime_column='DateTime',
        target_column="Corrected Upscaled Measurement [MW]",
        context_length=cf.model.context_length,
        frequency=cf.data.frequency,
        train_test_split_year=cf.data.train_test_split_year,
        train_val_split_year=cf.data.train_val_split_year,)

In [4]:
print(solar_dataset.data.shape)

torch.Size([3997, 1, 30])


In [5]:
df = solar_dataset.get_dataframe(preprocessed=True)

In [6]:
df.head()

,DateTime,Corrected Upscaled Measurement [MW]
0,2013-02-21,0.089717
1,2013-02-22,0.063666
2,2013-02-23,0.054708
3,2013-02-24,0.013538
4,2013-02-25,0.016952


In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [11]:
solar_dataset.train_val_split_index

2900

In [9]:

indices = list(range(len(solar_dataset)))
train_indices = indices[:solar_dataset.train_val_split_index]
val_indices = indices[solar_dataset.train_val_split_index:solar_dataset.train_test_split_index]
test_indices = indices[solar_dataset.train_test_split_index:]

# Creating data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = torch.utils.data.DataLoader(solar_dataset, batch_size=cf.model.batch_size, 
                                        sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(solar_dataset, batch_size=cf.model.batch_size,
                                                sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(solar_dataset, batch_size=cf.model.batch_size,sampler=test_sampler)

In [10]:
print(cf.model.encoder_type)

PositionalEncodingEncoder


In [11]:
# https://stackoverflow.com/questions/65996797/how-to-refresh-a-python-import-in-a-jupyter-notebook-cell
from importlib import reload
reload(building_blocks)
from building_blocks import *

model_params = TimeSeriesTransformerParams(
    input_dim=cf.model.context_length,
    embed_size=cf.model.embedding_size,
    num_layers=cf.model.num_layers,
    heads=cf.model.num_attention_heads,
    device=device,
    forward_expansion=cf.model.forward_expansion,
    dropout=cf.model.dropout,
    forecast_size=cf.model.forecast_size,
    encoder_type=cf.model.encoder_type,
    kernel_size=cf.model.kernel_size,
    padding_right=cf.model.padding_right
)
scenario_params = ScenarioParams(
    name="electricity",
    device=device,
    epochs=100,
    dataloader_train=train_loader,
    dataloader_validation=validation_loader,
    dataloader_test=test_loader,
    base_path="/dne" if cf.runtime.run_in_colab else "./"
)
model = TimeSeriesTransformer.from_params(model_params).to(device)
scenario = Scenario(scenario_params)
result = scenario.execute(model)
print(f"execution done")

c:\Users\Arne\anaconda3\envs\dne\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


checking paths, base path is ./
training and validating the model


c:\Users\Arne\anaconda3\envs\dne\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\Arne\anaconda3\envs\dne\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([22])) that is different to the input size (torch.Size([22, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


New best training score at epoch 1
Epoch 1/100, Training Loss: 6.2967, Validation Loss: nan


c:\Users\Arne\anaconda3\envs\dne\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Arne\anaconda3\envs\dne\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


New best training score at epoch 2
Epoch 2/100, Training Loss: 0.0251, Validation Loss: nan
New best training score at epoch 3
Epoch 3/100, Training Loss: 0.0250, Validation Loss: nan
New best training score at epoch 4
Epoch 4/100, Training Loss: 0.0236, Validation Loss: nan
New best training score at epoch 5
Epoch 5/100, Training Loss: 0.0229, Validation Loss: nan
New best training score at epoch 6
Epoch 6/100, Training Loss: 0.0226, Validation Loss: nan
New best training score at epoch 7
Epoch 7/100, Training Loss: 0.0215, Validation Loss: nan
Epoch 8/100, Training Loss: 0.0217, Validation Loss: nan
New best training score at epoch 9
Epoch 9/100, Training Loss: 0.0211, Validation Loss: nan
New best training score at epoch 10
Epoch 10/100, Training Loss: 0.0204, Validation Loss: nan
New best training score at epoch 11
Epoch 11/100, Training Loss: 0.0201, Validation Loss: nan
Epoch 12/100, Training Loss: 0.0202, Validation Loss: nan
New best training score at epoch 13
Epoch 13/100, Tra

KeyboardInterrupt: 